<a href="https://colab.research.google.com/github/chanyoung98/-/blob/main/5%EC%9E%A5_%ED%9A%8C%EC%9B%90_%ED%83%88%ED%87%B4%EB%A5%BC_%EC%98%88%EC%B8%A1%ED%95%98%EB%8A%94_%ED%85%8C%ED%81%AC%EB%8B%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [70]:
import pandas as pd
customer = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/파이썬실무테크닉/5장. 회원 탈퇴를 예측하는 테크닉/customer_join.csv")
uselog_months = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/파이썬실무테크닉/5장. 회원 탈퇴를 예측하는 테크닉/use_log_months.csv")

불과 몇 개월 만에 그만둔 회원도 많음 -> 과거 6개월분의 데이터를 이용한 예측은  의미가 없다. 따라서 이 달과 1개월 전의 이용 횟수를 집계한 데이터를 작성하자.

In [71]:
customer.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,종일,10500,일반,4.833333,5.0,8,2,1,2019-04-30,47
1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,종일,10500,일반,5.083333,5.0,7,3,1,2019-04-30,47
2,OA974876,XXXXX,C01,M,2015-05-01,NaN,CA1,0,종일,10500,일반,4.583333,5.0,6,3,1,2019-04-30,47
3,HD024127,XXXXX,C01,F,2015-05-01,NaN,CA1,0,종일,10500,일반,4.833333,4.5,7,2,1,2019-04-30,47
4,HD661448,XXXXX,C03,F,2015-05-01,NaN,CA1,0,야간,6000,일반,3.916667,4.0,6,1,1,2019-04-30,47


In [72]:
uselog_months.head()

,연월,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7


In [73]:
year_months = list(uselog_months['연월'].unique())
uselog = pd.DataFrame()
for i in range(1, len(year_months)):
  tmp = uselog_months.loc[uselog_months['연월'] == year_months[i]] #201805 행들만 뽑는다(tmp)
  tmp.rename(columns = {'count' : 'count_0'}, inplace = True) #tmp 현재 년도 수
  tmp_before = uselog_months.loc[uselog_months['연월'] == year_months[i-1]] #201804 행들만 뽑는다(tmp_before)
  del tmp_before['연월'] 
  tmp_before.rename(columns = {'count':'count_1'}, inplace = True) #tmp_before 전년도 수
  tmp = pd.merge(tmp, tmp_before, on = 'customer_id', how = 'left') #tmp와 tmp_before -> pd.merge
  uselog = pd.concat([uselog, tmp], ignore_index = True) #계속 concat
uselog.head()

/usr/local/lib/python3.9/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,연월,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


이제 탈퇴 전월의 데이터를 작성해보자.

# 42. 탈퇴 전월의 탈퇴 고객 데이터를 작성하자.
왜 end_date 칼럼의 탈퇴 월이 아닌 탈퇴 전월의 데이터를 작성할까?

탈퇴를 예측하는 목적은 탈퇴를 미연에 방지하기 위한 것이다. 이 스포츠 센터에선 월말까지 탈퇴 신청을 해야 다음 달 말에 탈퇴할 수 있다.  
ex. 2018년 9월에 탈퇴한 회원이 있다고 하면, 이 경우 8월에 탈퇴 신청을 했기에 9월 데이터를 사용한다면 탈퇴를 미연에 방지할 수 없다. 때문에 2018년 8월을 탈퇴 월로 하고, 그 1개월 전인 7월의 데이터로부터 8월에 탈퇴 신청을 할 확률을 예측하자.

먼저, 탈퇴한 회원을 추출하고 end_date의 1개월 전을 계산하여 연월에 저장 후 uselog, customer_id, 연월을 키로 결합한다.

In [74]:
from dateutil.relativedelta import relativedelta

exit_customer = customer.loc[customer['is_deleted'] == 1] #탈퇴한 사람들만 뽑음
exit_customer['exit_date'] = None #탈퇴 신청 날짜 칼럼을 만든다.
exit_customer['end_date'] = pd.to_datetime(exit_customer['end_date'])
for i in range(len(exit_customer)):
  exit_customer['exit_date'].iloc[i] = exit_customer['end_date'].iloc[i] - relativedelta(months = 1) #탈퇴 날짜 - 한 달 
exit_customer['exit_date'] = pd.to_datetime(exit_customer['exit_date'])
exit_customer['연월'] = exit_customer['exit_date'].dt.strftime("%Y%m") #탈퇴 전월을 연월로..?
uselog['연월'] = uselog['연월'].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on = ['customer_id', '연월'], how = 'left') #전체 uselog 중 탈퇴한 uselog를 연월 기준으로 뽑는다면,, uselog는 마지막 이용로그가 된다.
exit_uselog.head()

print(len(uselog))
print(len(exit_customer))

<ipython-input-74-5d060a3309a5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exit_customer['exit_date'] = None #탈퇴 날짜 칼럼을 만든다.
<ipython-input-74-5d060a3309a5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exit_customer['end_date'] = pd.to_datetime(exit_customer['end_date'])
/usr/local/lib/python3.9/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/panda

33851
1350


결합한 데이터는 탈퇴한 회원의 탈퇴 전월의 데이터뿐이므로 결측치가 많다. 결측치가 없는 데이터만 남기고 나머지를 제거하자.

In [75]:
exit_uselog = exit_uselog.dropna(subset = ['name'])
print(len(exit_uselog)) #33854 -> 1104
print(len(exit_uselog['customer_id'].unique())) 
exit_uselog.head()

1104
1104


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,일반,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,일반,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,일반,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30


지금껏 확인했듯이, customer 데이터는 end_date 이외에 결측치는 없다. exit_uselog의 데이터 수와 customer_id의 유니크한 개수를 세어보면 일치한다.  
지금껏 작성한 데이터는 어떤 특정 회원이 그만두기 전월의 상태를 나타내는 데이터이다.

지금껏 탈퇴회원의 데이터를 작성하였다. 이제 탈퇴하지 않은 지속 회원의 데이터도 작성해보자.

# 지속 회원의 데이터를 작성하자.

지속 회원은 탈퇴 월이 없기에 어떤 연월의 데이터를 작성해도 된다. 따라서 지속회원을 추출한 후 uselog 데이터에 결합하면 작성된다.

In [76]:
customer.head(5)

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,종일,10500,일반,4.833333,5.0,8,2,1,2019-04-30,47
1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,종일,10500,일반,5.083333,5.0,7,3,1,2019-04-30,47
2,OA974876,XXXXX,C01,M,2015-05-01,NaN,CA1,0,종일,10500,일반,4.583333,5.0,6,3,1,2019-04-30,47
3,HD024127,XXXXX,C01,F,2015-05-01,NaN,CA1,0,종일,10500,일반,4.833333,4.5,7,2,1,2019-04-30,47
4,HD661448,XXXXX,C03,F,2015-05-01,NaN,CA1,0,야간,6000,일반,3.916667,4.0,6,1,1,2019-04-30,47


In [77]:
uselog.head(3)

,연월,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN


In [78]:
conti_customer = customer.loc[customer['is_deleted'] == 0] #탈퇴하지 않은 고객들
conti_uselog = pd.merge(uselog, conti_customer, on = ['customer_id'], how = 'left')
print(len(conti_uselog))
conti_uselog = conti_uselog.dropna(subset = ['name']) #name 컬럼에서 결측치 제거
print(len(conti_uselog))

33851
27422


uselog와의 결합은 customer_id로만 한다. 앞서 말한 것처럼 탈퇴 월이 없기에 2018년 5월 A씨의 데이터나 2018년 12월 A씨의 데이터, 아무거나 지속 고객의 데이터로 사용할 수 있다.  

또한 name 칼럼의 결손 데이터를 제거하고 탈퇴 회원을 제거한다. 이로써 데이터가 33851개에서 27422개로 줄어들었다.

탈퇴 데이터가 1104개이기에 지속회원 데이터 27422를 모두 사용한다면 불균형 데이터가 된다. 

간단히 지속 회원 데이터도 회원당 개가 되게 언더샘플링하자. 다시 말해, 2018년 5월 A씨와 2018년 12월 A씨 중 하나만 선택한다는 뜻이다.

이를 위해 데이터를 섞고 중복을 제거하는 방법을 이용한다.

In [79]:
conti_uselog.head()

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201805,AS002855,5,4.0,XXXX,C03,F,2016-11-01,NaN,CA1,...,야간,6000.0,일반,4.500000,5.0,7.0,2.0,1.0,2019-04-30,29.0
1,201805,AS009373,4,3.0,XX,C01,F,2015-11-01,NaN,CA1,...,종일,10500.0,일반,5.083333,5.0,7.0,3.0,1.0,2019-04-30,41.0
2,201805,AS015233,7,NaN,XXXXX,C01,M,2018-05-13,NaN,CA2,...,종일,10500.0,입회비반액할인,7.545455,7.0,11.0,4.0,1.0,2019-04-30,11.0
3,201805,AS015315,3,6.0,XXXXX,C01,M,2015-07-01,NaN,CA1,...,종일,10500.0,일반,4.833333,5.0,7.0,3.0,1.0,2019-04-30,45.0
4,201805,AS015739,5,7.0,XXXXX,C03,M,2017-06-01,NaN,CA1,...,야간,6000.0,일반,5.583333,5.5,8.0,4.0,1.0,2019-04-30,22.0


In [80]:
conti_uselog = conti_uselog.sample(frac = 1).reset_index(drop = True) #sample(frac = 1) : 100% row를 뽑아서 샘플링하고, 인덱스 배열 정리
conti_uselog = conti_uselog.drop_duplicates(subset = 'customer_id') #customer_id를 기준으로 중복 행 제거
#2018년 5월 A씨와 2018년 12월 A씨 중 하나가 랜덤으로 선택
print(len(conti_uselog))
conti_uselog.head()

2842


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201812,HD574185,3,2.0,XXXXX,C02,M,2016-09-01,NaN,CA1,...,주간,7500.0,일반,4.416667,4.5,7.0,2.0,1.0,2019-04-30,31.0
1,201809,IK755601,6,8.0,XXXX,C03,F,2017-06-01,NaN,CA1,...,야간,6000.0,일반,6.000000,6.0,8.0,4.0,1.0,2019-04-30,22.0
2,201901,TS181112,5,6.0,XXXXX,C03,F,2017-07-01,NaN,CA2,...,야간,6000.0,입회비반액할인,6.000000,6.0,8.0,4.0,1.0,2019-04-30,21.0
3,201809,OA470195,6,6.0,XXXX,C01,M,2016-06-01,NaN,CA1,...,종일,10500.0,일반,5.333333,5.0,8.0,3.0,1.0,2019-04-30,34.0
4,201811,PL499733,3,4.0,XXXX,C02,F,2017-10-01,NaN,CA1,...,주간,7500.0,일반,6.166667,6.0,11.0,3.0,1.0,2019-04-30,18.0


이로서 지속 회원 데이터와 탈퇴 회원 데이터가 작성되었다. 이를 세로로 결합해보자.

In [81]:
predict_data = pd.concat([conti_uselog, exit_uselog], ignore_index = True) #ignore_index = True를 함으로써 기존 데이터의 인덱스를 무시
print(len(predict_data))
predict_data.head()

3946


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201812,HD574185,3,2.0,XXXXX,C02,M,2016-09-01,NaN,CA1,...,7500.0,일반,4.416667,4.5,7.0,2.0,1.0,2019-04-30,31.0,NaT
1,201809,IK755601,6,8.0,XXXX,C03,F,2017-06-01,NaN,CA1,...,6000.0,일반,6.000000,6.0,8.0,4.0,1.0,2019-04-30,22.0,NaT
2,201901,TS181112,5,6.0,XXXXX,C03,F,2017-07-01,NaN,CA2,...,6000.0,입회비반액할인,6.000000,6.0,8.0,4.0,1.0,2019-04-30,21.0,NaT
3,201809,OA470195,6,6.0,XXXX,C01,M,2016-06-01,NaN,CA1,...,10500.0,일반,5.333333,5.0,8.0,3.0,1.0,2019-04-30,34.0,NaT
4,201811,PL499733,3,4.0,XXXX,C02,F,2017-10-01,NaN,CA1,...,7500.0,일반,6.166667,6.0,11.0,3.0,1.0,2019-04-30,18.0,NaT


이것으로 유지 회원과 탈퇴 회원이 섞인 기본 데이터를 작성하였다. 이제 변수를 추가해보자.

# 44. 예측할 달의 재적 기간을 작성하자.

시간적 요소가 들어간 데이터이기에 재적 기간과 같은 데이터를 변수로 이용하는 것은 좋은 접근이다. 따라서 재적기간 열을 추가한다.

In [82]:
predict_data['period'] = 0 #재적기간
predict_data['now_date'] = pd.to_datetime(predict_data['연월'], format = "%Y%m")
predict_data['start_date'] = pd.to_datetime(predict_data['start_date'])

for i in range(len(predict_data)):
  delta = relativedelta(predict_data['now_date'][i], predict_data['start_date'][i])
  predict_data['period'][i] = int(delta.years*2 + delta.months) #years에 2 곱하기...?
predict_data.head()

<ipython-input-82-69fe9e18beb7>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_data['period'][i] = int(delta.years*2 + delta.months) #years에 2 곱하기...?


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201812,HD574185,3,2.0,XXXXX,C02,M,2016-09-01,NaN,CA1,...,4.416667,4.5,7.0,2.0,1.0,2019-04-30,31.0,NaT,7,2018-12-01
1,201809,IK755601,6,8.0,XXXX,C03,F,2017-06-01,NaN,CA1,...,6.000000,6.0,8.0,4.0,1.0,2019-04-30,22.0,NaT,5,2018-09-01
2,201901,TS181112,5,6.0,XXXXX,C03,F,2017-07-01,NaN,CA2,...,6.000000,6.0,8.0,4.0,1.0,2019-04-30,21.0,NaT,8,2019-01-01
3,201809,OA470195,6,6.0,XXXX,C01,M,2016-06-01,NaN,CA1,...,5.333333,5.0,8.0,3.0,1.0,2019-04-30,34.0,NaT,7,2018-09-01
4,201811,PL499733,3,4.0,XXXX,C02,F,2017-10-01,NaN,CA1,...,6.166667,6.0,11.0,3.0,1.0,2019-04-30,18.0,NaT,3,2018-11-01


# 결측치를 제거하자
먼저 결측치 수를 파악하자

In [83]:
predict_data.isna().sum()

연월                      0
customer_id             0
count_0                 0
count_1               281
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

count_1이 결손된 데이터만 일단 제거하자.

In [84]:
predict_data = predict_data.dropna(subset = ['count_1'])
predict_data.isnull().sum()

연월                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2613
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2613
period                  0
now_date                0
dtype: int64

이것으로 결측치 처리를 완료하였다. 이제 머신러닝을 위한 마지막 데이터 처리를 다루어보자.

# 46. 문자열 변수를 처리할 수 있게 가공하자.
가입 캠페인 구분, 회원 구분 등과 같은 문자열 데이터를 활용하기 위해서는 앞에서 routine_flg를 작성한 것 처럼 플래그를 만든다. 이를 더미 변수라고 한다.

우선, 이번 예측에 사용할 데이터를 추출해보자  
설명변수
- 1개월 전의 이용 횟수 count_1
- 카테고리 변수인 campaign_name, class_name, gender
- 정기 이용 여부 플래그 routine_flg
- 재적 기간 period  

목적변수
- is_deleted

지도학습의 분류로 진행해보자. 분류는 회귀와는 달리, 탈퇴/유지와 같은 이산치를 목적 변수로 사용한다.

In [85]:
target_col = ['campaign_name', 'class_name', 'gender', 'count_1', 'routine_flg', 'period', 'is_deleted']
predict_data = predict_data[target_col]
predict_data.head()

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,일반,주간,M,2.0,1.0,7,0.0
1,일반,야간,F,8.0,1.0,5,0.0
2,입회비반액할인,야간,F,6.0,1.0,8,0.0
3,일반,종일,M,6.0,1.0,7,0.0
4,일반,주간,F,4.0,1.0,3,0.0


이제 카테고리 변수를 이용해서 더미 변수를 만들어보자.

In [86]:
predict_data = pd.get_dummies(predict_data)
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_일반,campaign_name_입회비무료,campaign_name_입회비반액할인,class_name_야간,class_name_종일,class_name_주간,gender_F,gender_M
0,2.0,1.0,7,0.0,1,0,0,0,0,1,0,1
1,8.0,1.0,5,0.0,1,0,0,1,0,0,1,0
2,6.0,1.0,8,0.0,0,0,1,1,0,0,1,0
3,6.0,1.0,7,0.0,1,0,0,0,1,0,0,1
4,4.0,1.0,3,0.0,1,0,0,0,0,1,1,0


더미 변수에서는 한 가지 주의가 필요하다. 각 더미 변수에서 하나씩 지워야 한다.

In [87]:
del predict_data['campaign_name_일반']
del predict_data['class_name_야간']
del predict_data['gender_F']
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_입회비무료,campaign_name_입회비반액할인,class_name_종일,class_name_주간,gender_M
0,2.0,1.0,7,0.0,0,0,0,1,1
1,8.0,1.0,5,0.0,0,0,0,0,0
2,6.0,1.0,8,0.0,0,1,0,0,0
3,6.0,1.0,7,0.0,0,0,1,0,1
4,4.0,1.0,3,0.0,0,0,0,1,0


# 47. 의사결정 트리를 사용해서 탈퇴 예측 모델을 구축하자.



In [88]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection
exit = predict_data.loc[predict_data['is_deleted'] == 1]
conti = predict_data.loc[predict_data['is_deleted'] == 0].sample(len(exit))

X = pd.concat([exit, conti], ignore_index = True)
y = X['is_deleted']
del X['is_deleted']
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)

model = DecisionTreeClassifier(random_state = 0)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(y_test_pred)

[0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0.
 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1.
 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0.
 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1.
 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1.
 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0.
 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1.
 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0.
 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0.
 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0.
 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 0.
 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0.
 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0.

실제 정답과의 비교를 위해 실제 값 y_test와 함께 데이터프레임에 저장해두자.

In [89]:
results_test = pd.DataFrame({"y_test" : y_test, "y_pred" : y_test_pred})
results_test.head()

,y_test,y_pred
2103,0.0,0.0
1415,0.0,0.0
1912,0.0,0.0
1750,0.0,0.0
362,1.0,0.0


# 예측 모델을 평가하고 모델을 튜닝해보자.

우선 앞에서 작성한 results_test 데이터를 집계하여 정답률을 계산해보자. 정답 수는 results_test 데이터의 y_test와 y_pred가 일치하는 개수이다. 해당 개수를 전체 데이터 수로 나누면 정답률이다.

In [90]:
correct = len(results_test.loc[results_test['y_test'] == results_test['y_pred']])
data_count = len(results_test)
score_test = correct/data_count
print(score_test)

0.8098859315589354


이 정확도를 그대로 받아들일 수 없다. 머신러닝의 목적은 어디까지나 미지의 데이터를 이용한 예측이며, 학습용 데이터로 예측한 정확도와 평가용 데이터로 예측한 정확도의 차이가 작은 것이 이상적일 것이다.

In [91]:
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.8098859315589354
0.9334600760456274


출력 결과를 보면, 학습용 데이터에 너무 맞춘 과적합 경향이 있다. 이런 경우 데이터 늘리기, 변수 재검토, 모델의 파라미터 변경과 같은 방법을 적용하여 이상적인 모델로 만들어 갈 수 있다.   

여기서는 모델의 파라미터를 바꿔보자.

In [92]:
X = pd.concat([exit, conti], ignore_index = True)
y = X['is_deleted']
del X['is_deleted']
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)

model = DecisionTreeClassifier(random_state = 0, max_depth = 5)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.7965779467680608
0.829531051964512


In [102]:
from sklearn.model_selection import GridSearchCV
model = DecisionTreeClassifier()
parameters = {'max_depth' : [5,7,9],
              'min_samples_split' : [2,4,6],
              'random_state' : [0]}
clf = GridSearchCV(model, parameters)
clf.fit(X_train, y_train)

print(clf.best_params_)
print(clf.best_score_)

{'max_depth': 7, 'min_samples_split': 4, 'random_state': 0}
0.8599477596945952


In [103]:
model = DecisionTreeClassifier(random_state = 0, max_depth = 7, min_samples_split = 4)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.8593155893536122
0.8783269961977186


과적합 여부가 상대적으로 줄어들었음을 확인할 수 있다. max_depth를 지정하면 모델이 단순해져 미지의 데이터에도 대응할 수 있는 좋은 모델이 된다.

# 모델에 기여하는 변수를 확인하자.

In [ ]:
importance = pd.DataFrame({'feature_names' : X.columns, 'coefficient' : model.feature_importances_})
importance

model.feature_importances_로 변수 중요도를 얻을 수 있었다.  

마지막으로 변수를 작성해서 예측해보자.

# 회원 탈퇴를 예측하자.

모델을 작성할 때 사용한 설명 변수를 참고하여 예측을 위한 데이터를 작성하자.

In [52]:
count_1 = 3
routing_flg = 1
period = 10
campaign_name = '입회비무료'
class_name = '종일'
gender = 'M'

변수가 정의되면 데이터를 가공한다.

In [53]:
if campaign_name == '입회비반값할인':
  campaign_name_list = [1,0]
elif campaign_name == '입회비무료':
  campaign_name_list = [0,1]
elif campaign_name == '일반':
  campaign_name_list = [0,0]
if class_name == '종일':
  class_name_list = [1,0]
elif class_name == '주간':
  class_name_list = [0,1]
elif class_name == '야간':
  class_name_list = [0,0]
if gender == 'F':
  gender_list = [1]
elif gender == 'M':
  gender_list = [0]

input_data = [count_1, routing_flg, period]
input_data.extend(campaign_name_list)
input_data.extend(class_name_list)
input_data.extend(gender_list)

In [54]:
print(model.predict([input_data]))
print(model.predict_proba([input_data]))

[1.]
[[0.12149533 0.87850467]]


/usr/local/lib/python3.9/dist-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


1행에서 예측 분류 결과를 보면 1, 즉 탈퇴가 예상된다.  
2행에서는 0.95%의 확률로 탈퇴를 예측한다.